## Automatic Speech Recognition Inference using Online Endpoints

This sample shows how to deploy `automatic-speech-recognition` type models to an online endpoint for inference.

### Task
`automatic-speech-recognition` (ASR) converts a speech signal to text, mapping a sequence of audio inputs to text outputs. Virtual assistants like Siri and Alexa use ASR models to help users everyday, and there are many other useful user-facing applications like live captioning and note-taking during meetings.

### Model
Models that can perform the `automatic-speech-recognition` task are tagged with `task: automatic-speech-recognition`. We will use the `openai-whisper-large` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import-model-from-huggingface.ipynb) and then use them for inference. 

### Inference data
We will use custom audio files that have been uploaded to the cloud. \
You can replace the links with any audio file stored on the cloud and verify inference.
- Most common audio formats (m4a, wav, flac, wma, mp3, etc.) are supported.
- The whisper model can process only 30 seconds of data at a time, so if the file you upload is longer than 30 seconds, only the first 30 seconds will be transcribed. This can be circumvented by splitting the file into 30 second chunks.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for real time inference.
* Test the endpoint
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
    ClientSecretCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id="ea4faa5b-5e44-4236-91f6-5483d5b17d14",
    resource_group_name="amyharrispersonal",
    workspace_name="amyharris-canary",
)
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `automatic-speech-recognition` task. In this example, we use the `openai-whisper-large` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [2]:
model_name = "openai-whisper-large"
model_version = "1"
foundation_model = registry_ml_client.models.get(model_name, model_version)
print(
    "\n\nUsing model name: {0}, version: {1}, id: {2} for inferencing".format(
        foundation_model.name, foundation_model.version, foundation_model.id
    )
)



Using model name: openai-whisper-large, version: 1, id: azureml://registries/azureml-preview/models/openai-whisper-large/versions/1 for inferencing


### 3. Deploy the model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [3]:
import time, sys
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    OnlineRequestSettings,
)

# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "asr-" + str(timestamp)
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for automatic-speech-recognition task",
    auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [4]:
# create a deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS4_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=60000,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {"demo": 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

Check: endpoint asr-1684194834 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


......................................................................................................................

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://asr-1684194834.eastus2euap.inference.ml.azure.com/score', 'openapi_uri': 'https://asr-1684194834.eastus2euap.inference.ml.azure.com/swagger.json', 'name': 'asr-1684194834', 'description': 'Online endpoint for openai-whisper-large, for automatic-speech-recognition task', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/resourcegroups/amyharrispersonal/providers/microsoft.machinelearningservices/workspaces/amyharris-canary/onlineendpoints/asr-1684194834', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/ea4faa5b-5e44-4236-91f6-5483d5b17d14/providers/Microsoft.MachineLearningServices/locations/eastus2euap/mfeOperationsStatus/oe:c76e6446-545b-4141-80f9-e8ad59c471f2:1a4530e8-f9ee-47da-98bf-3f0e63bbb38f?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/ea4faa5b-5e44-42

### 4. Test the endpoint with sample data

We will fetch the sample scoring request and submit to online endpoint for inference. We will then show the display the scored labels alongside the ground truth labels

In [5]:
import pandas as pd

# score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file="./sample-request/sample_score.json",
)
print("raw response: \n", response, "\n")
# convert the json response to a pandas dataframe
response_df = pd.read_json(response)
response_df.head()

raw response: 
 [{"text": " This is a bright day."}, {"text": " This is a bright day."}] 



,text
0,This is a bright day.
1,This is a bright day.


### 5. Delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint

In [6]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

..........................................................................................